In [1]:
test_input = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533"""

test_input2 = """111111111111
999999999991
999999999991
999999999991
999999999991"""

In [2]:
input = open("inputs/17").read()

In [3]:
import numpy as np
import math
import networkx as nx

def parse_board(input):
    return np.array(list(map(list, input.splitlines())))

In [4]:
# grid = parse_board(test_input).astype(int)
# grid = parse_board(test_input2).astype(int)

grid = parse_board(input).astype(int)

In [5]:
grid

array([[1, 2, 2, ..., 2, 1, 2],
       [2, 2, 2, ..., 1, 2, 1],
       [1, 2, 2, ..., 1, 2, 1],
       ...,
       [1, 2, 1, ..., 1, 1, 2],
       [1, 1, 1, ..., 1, 2, 1],
       [2, 2, 2, ..., 2, 1, 2]])

In [6]:
# first going to code without the memory assumption

In [7]:
directions = {
    "right": (0, 1),
    "left": (0, -1),
    "up": (-1, 0),
    "down": (1, 0)
}

In [8]:
turn_left = {'left': 'down', 'down': 'right', 'right': 'up', 'up': 'left'}
turn_right = {'left': 'up', 'up': 'right', 'right': 'down', 'down': 'left'}

In [9]:
grid

array([[1, 2, 2, ..., 2, 1, 2],
       [2, 2, 2, ..., 1, 2, 1],
       [1, 2, 2, ..., 1, 2, 1],
       ...,
       [1, 2, 1, ..., 1, 1, 2],
       [1, 1, 1, ..., 1, 2, 1],
       [2, 2, 2, ..., 2, 1, 2]])

In [10]:
def in_bounds(shape, pos):
    return all(0 <= pos[i] < shape[i] for i in range(len(shape)))

def add_tuples(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))

In [11]:
destination = (grid.shape[0]-1, grid.shape[1]-1)
destination

(140, 140)

In [12]:
# our nodes are going to have (position, dir, straight_steps) as a tuple
# so if the board side length is N
# we will have roughly N^2 * 4 * 3 nodes

# you have max 3 options at each node: go straight, turn left, turn right. so the number of edges is ~ 3 * N^2 * 4 * 3

In [13]:
import itertools

g = nx.DiGraph()

def add_edges(current_node):
    pos, dir, straight_steps = current_node

    straight_pos = add_tuples(pos, directions[dir])
    if straight_steps < 3 and in_bounds(grid.shape, straight_pos):
        g.add_edge(current_node, (straight_pos, dir, straight_steps+1), cost=grid[straight_pos])

    left_pos = add_tuples(pos, directions[turn_left[dir]])
    if in_bounds(grid.shape, left_pos):
        g.add_edge(current_node, (left_pos, turn_left[dir], 1), cost=grid[left_pos])

    right_pos = add_tuples(pos, directions[turn_right[dir]])
    if in_bounds(grid.shape, right_pos):
        g.add_edge(current_node, (right_pos, turn_right[dir], 1), cost=grid[right_pos])

dirs = ['right', 'left', 'up', 'down']

for current_node in itertools.product(itertools.product(*map(range, grid.shape)), dirs, [1, 2, 3]):
    add_edges(current_node)

# special case for the starting position
for dir in dirs:
    add_edges(((0, 0), dir, 0))

In [14]:
g.number_of_edges(), g.number_of_nodes()

(631686, 238576)

In [15]:
# min(walk((0, 1), 'right', 0, 0), walk((1, 0), 'down', 0, 0))

source_nodes = [n for n in g.nodes() if n[0] == (0, 0) and n[-1] == 0 and n[-2] in ['down', 'right']]

In [16]:
source_nodes

[((0, 0), 'right', 0), ((0, 0), 'down', 0)]

In [17]:
dest_nodes = [n for n in g.nodes() if n[0] == destination and n[-2] in ['down', 'right']]

In [18]:
dest_nodes

[((140, 140), 'down', 1),
 ((140, 140), 'down', 2),
 ((140, 140), 'down', 3),
 ((140, 140), 'right', 2),
 ((140, 140), 'right', 3),
 ((140, 140), 'right', 1)]

In [19]:
# list(g.edges(data=True))

In [20]:
from tqdm import tqdm

min_cost = math.inf

for sn, dn in tqdm(itertools.product(source_nodes, dest_nodes)):
    try:
        c = nx.dijkstra_path_length(g, sn, dn, weight='cost')

        if c < min_cost:
            min_cost = c
    except Exception as e:
        pass

min_cost

12it [00:07,  1.59it/s]


638

now pt2


In [21]:
g = nx.DiGraph()

def add_edges(current_node):
    pos, dir, straight_steps = current_node

    straight_pos = add_tuples(pos, directions[dir])

    # if we haven't gone at least 4 then we have to go straight, if possible
    if straight_steps < 4:
        if in_bounds(grid.shape, straight_pos):
            g.add_edge(current_node, (straight_pos, dir, straight_steps+1), cost=grid[straight_pos])
    else:
        # otherwise, if we've gone less than 10 we have the option to go straight
        if straight_steps < 10 and in_bounds(grid.shape, straight_pos):
            g.add_edge(current_node, (straight_pos, dir, straight_steps+1), cost=grid[straight_pos])

        left_pos = add_tuples(pos, directions[turn_left[dir]])
        if in_bounds(grid.shape, left_pos):
            g.add_edge(current_node, (left_pos, turn_left[dir], 1), cost=grid[left_pos])

        right_pos = add_tuples(pos, directions[turn_right[dir]])
        if in_bounds(grid.shape, right_pos):
            g.add_edge(current_node, (right_pos, turn_right[dir], 1), cost=grid[right_pos])

dirs = ['right', 'left', 'up', 'down']

for current_node in itertools.product(itertools.product(*map(range, grid.shape)), dirs, list(range(1, 11))):
    add_edges(current_node)

# special case for the starting position
for dir in dirs:
    add_edges(((0, 0), dir, 0))

In [22]:
g.number_of_edges()

1816082

In [23]:
source_nodes = [n for n in g.nodes() if n[0] == (0, 0) and n[-1] == 0 and n[-2] in ['down', 'right']]
dest_nodes = [n for n in g.nodes() if n[0] == destination and n[-2] in ['down', 'right'] and n[-1] >= 4]

In [24]:
len(source_nodes), len(dest_nodes)

(2, 14)

In [25]:
from tqdm import tqdm

min_cost = math.inf

for sn, dn in tqdm(itertools.product(source_nodes, dest_nodes)):
    try:
        c = nx.dijkstra_path_length(g, sn, dn, weight='cost')

        if c < min_cost:
            min_cost = c
    except Exception as e:
        pass

min_cost

28it [01:12,  2.60s/it]


748